# Car Charger Project

In [2]:
import threading
import paho.mqtt.client as mqtt

from stmpy import Machine, Driver

import ipywidgets as widgets
from IPython.display import display

ModuleNotFoundError: No module named 'paho'

In [27]:
broker, port = "broker.hivemq.com", 1883

# Publish
charger = mqtt.Client(callback_api_version=mqtt.CallbackAPIVersion.VERSION1)
charger.connect(broker, port)

def chargerAvail():
    charger.publish("chargerArea", "Charger is available")

def chargerBusy():
    charger.publish("chargerArea", "Charger is occupied")

def chargerBroken():
    charger.publish("chargerArea", "Charger is broken")

def chargerFix():
    charger.publish("chargerArea", "Charger is fixed")

def chargerPercent(percentage):
    charger.publish("chargerArea", f"Car battery {percentage}% charged")

# Subscriber
def subscribe_mqtt():
    subscriber = mqtt.Client(callback_api_version=mqtt.CallbackAPIVersion.VERSION1)
    subscriber.connect(broker, port)
    subscriber.subscribe("chargerArea")
    subscriber.loop_forever()

subscriber_thread = threading.Thread(target=subscribe_mqtt)
subscriber_thread.daemon = True  # Thread will exit when the main thread exits
subscriber_thread.start()

C:\Users\danie\AppData\Local\Temp\ipykernel_8472\1876157454.py:4: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  charger = mqtt.Client(callback_api_version=mqtt.CallbackAPIVersion.VERSION1)
C:\Users\danie\AppData\Local\Temp\ipykernel_8472\1876157454.py:24: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  subscriber = mqtt.Client(callback_api_version=mqtt.CallbackAPIVersion.VERSION1)


In [28]:
class Car:
    def __init__(self):
        self.carID = "chargerUser"
        self.batteryPercentage = 20
        self.isCharging = False
        self.invoice = 0

        self.toogleButton = widgets.ToggleButtons(
            options=['Owner', 'Visitor'],
        )
        #Visualize relevant car info
        self.bool = widgets.Button(description="Car Battery:")
        self.bool2 = widgets.Button(description=str(self.batteryPercentage)+"%")
        self.chargeBar = widgets.IntProgress(value=self.batteryPercentage,
                                         min=0,
                                         max=100,
                                         description='Battery:',
                                         bar_style='success', # 'success', 'info', 'warning', 'danger' or ''
                                         style={'bar_color': 'green'},
                                         orientation='horizontal')
        
        self.bool3 = widgets.Button(description="Invoice")
        self.bool4 = widgets.Button(description=str(self.invoice)+"kr")

        empty_widget = widgets.Label(value="")
        items = [self.toogleButton, empty_widget, empty_widget, self.bool,self.bool2,self.chargeBar, self.bool3, self.bool4]
        Box = widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(3, 150px)"))

        #Format in tabs
        tab = widgets.Tab()
        tab.children = [Box]
        tab.titles = ['Car: '+self.carID]
        display(tab)


    def switch_state(self, on):
        self.invoice = 0
        self.toogleButton.disabled = False
        if on: self.isCharging = True
        else: self.isCharging = False


    def ChargeCar(self):
        self.toogleButton.disabled = True
        if (self.isCharging and self.batteryPercentage<100) :
            if self.toogleButton.value == 'Visitor':
                self.invoice += 3
            self.batteryPercentage += 1

            #For MQTT
            if self.batteryPercentage % 10 == 0:
                chargerPercent(self.batteryPercentage)
                
            self.bool2.set_trait(name='description', value=str(self.batteryPercentage)+"%")
            self.bool4.set_trait(name='description', value=str(self.invoice)+"kr")
            self.chargeBar.set_trait(name='value', value=self.batteryPercentage)
        elif self.batteryPercentage>=100:
            self.chargeBar.set_trait(name='description', value='Full:')


    def set_charge(self, on):
        if (on and self.batteryPercentage<100):
            self.chargeBar.set_trait(name='description', value='Charging:')
        else:
            self.chargeBar.set_trait(name='description', value='Not charging:')

In [29]:
class Charger:
    def __init__(self):
    
        self.ChargerID = "charger"
        self.IsOccupied = False 
        self.IsBroken = False
    
      # load images and store them
        self.plug_on = open("images/timer/plug_on.jpg", "rb").read()
        self.plug_off = open("images/timer/plug_off.jpg", "rb").read()

        self.broken = open("images/red_on.png", "rb").read()
        self.fixed = open("images/green_on.png", "rb").read()
        self.plug = widgets.Image(value=self.plug_off, format='jpg', width=100, height=100)
        self.status = widgets.Image(value=self.fixed, format='png', width=100, height=100)

        #vizualize relevant info
        self.button = widgets.Button(description="Plug in charger")
        self.button.on_click(self.on_charger_plugin)
        self.button2 = widgets.Button(description="Plug out charger")
        self.button2.on_click(self.on_charger_plugout)
        self.button3 = widgets.Button(description="break")
        self.button3.on_click(self.on_charger_broken)
        self.button4 = widgets.Button(description="fix")
        self.button4.on_click(self.on_charger_fixed)

        self.bool1 = widgets.Button(description="status {busy or idle}:")
        self.bool2 = widgets.Button(description="Status {broken or not}:")

        items1 = [self.button, self.button2, self.button3, self.button4,self.bool1,self.plug, self.bool2, self.status]
        buttonBox = widgets.GridBox(items1, layout=widgets.Layout(grid_template_columns="repeat(4, 150px)"))
        
        #Format in tabs
        tab = widgets.Tab()
        tab.children = [buttonBox]
        tab.titles = ['Charger: '+self.ChargerID]
        display(tab)


    def on_charger_plugin(self, b):
        if (not self.IsBroken and not self.IsOccupied):
            self.IsOccupied = True
            self.stm.send('plug_in') # <---- here we send a signal
            self.stm.driver.send('plugIn', 'stm_car')
            #For MQTT
            chargerBusy()
        
    def on_charger_plugout(self, b):
        if self.IsOccupied:
            self.IsOccupied = False
            self.stm.send('plug_out') # <---- here we send a signal
            self.stm.driver.send('plugOut', 'stm_car')
            #For MQTT
            chargerAvail()
        
    def on_charger_broken(self, b):
        if not self.IsBroken:
            self.IsBroken = True
            self.stm.send('break') # <---- here we send a signal
            #For MQTT
            chargerBroken()
            if self.IsOccupied:
                self.stm.driver.send('plugOut', 'stm_car')
                self.IsOccupied = False
       
        
    def on_charger_fixed(self, b):
        if self.IsBroken:
            self.IsBroken = False
            self.stm.send('fix') # <---- here we send a signal
            #For MQTT
            chargerFix()


    def switch_plug(self, on):
        if on: self.plug.set_trait(name='value', value=self.plug_on)
        else: self.plug.set_trait(name='value', value=self.plug_off)
            
    def switch_status(self, on):
        if on: self.status.set_trait(name='value', value=self.fixed)
        else: self.status.set_trait(name='value', value=self.broken)


In [30]:
carCharger = Charger()
car = Car()

t0 = {'source': 'initial',
      'target': 'available'}

t1 = {'trigger':'plug_in',
      'source':'available',
      'target': 'busy',
      'effect':'switch_plug(True)'
     }

t2 = {'trigger':'plug_out',
      'source':'busy',
      'target': 'available',
      'effect':'switch_plug(False)'
     }

t3 = {'trigger':'break',
      'source': 'available',
      'target': 'broken',
      'effect':'switch_plug(False);switch_status(False)'
     }

t4 = {'trigger':'fix',
      'source':'broken',
      'target': 'available',
      'effect':'switch_status(True)'
     }

t5 = {'trigger':'break',
      'source':'busy',
      'target': 'broken',
      'effect':'switch_status(False);switch_plug(False)'
     }


#car transitions

t6 = {'source': 'initial',
      'target': 'CarNotCharging'}

t7 = {'trigger':'plugIn',
      'source':'CarNotCharging',
      'target': 'CarCharging',
      'effect':'switch_state(True);start_timer("chargeTimer",1000);set_charge(True)',
     }
t8 = {'trigger':'plugOut',
      'source':'CarCharging',
      'target': 'CarNotCharging',
      'effect':'switch_state(False);set_charge(False);stop_timer("chargeTimer")'
     }
t10 = {'trigger':'chargeTimer',
      'source':'CarCharging',
      'target': 'CarCharging',
      'effect':'ChargeCar();start_timer("chargeTimer",1000)'
     }


stm_carCharger = Machine(name='stm_carCharger', transitions=[t0, t1,t2,t3,t4,t5], obj=carCharger)
carCharger.stm = stm_carCharger
stm_car = Machine(name='stm_car', transitions=[t6, t7, t8, t10], obj=car)
car.stm = stm_car


driver = Driver()
driver.add_machine(stm_carCharger)
driver.add_machine(stm_car)

driver.start()
